In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier  # ou KNeighborsRegressor si c'est une régression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances
from sklearn.preprocessing import MinMaxScaler
from numpy import zeros

In [3]:
df = pd.read_csv("df_sans_vec.csv")

In [4]:
#def filtre_role(role, nom):
#    return df3[df3[f"{role}_{nom}"]==1]


In [5]:
df = df.set_index(df["originalTitle"]+"_"+df["startYear"].astype(str))

In [6]:
compteur_real = {}
compteur_acteurs =  {}
compteur_director = {}
compteur_composer = {}
compteur_writer = {}
for i, row in df.iterrows():
  try:
    names = eval(row["noms"])
    jobs = eval(row["jobs"])
    for name, job in zip(names, jobs):
      if job == "actress":
        if name not in compteur_acteurs:
            compteur_acteurs[name] = 1
        else:
            compteur_acteurs[name]+=1
      if job == "actor":
        if name not in compteur_acteurs:
          compteur_acteurs[name] = 1
        else:
          compteur_acteurs[name]+=1
      if job == "director":
        if name not in compteur_director:
          compteur_director[name] = 1
        else:
          compteur_director[name]+=1
      if job == "real":
        if name not in compteur_real:
          compteur_real[name] = 1
        else:
          compteur_real[name]+=1
      if job == "writer":
        if name not in compteur_writer:
          compteur_writer[name] = 1
        else:
          compteur_writer[name]+=1
      if job == "composer":
        if name not in compteur_composer:
          compteur_composer[name] = 1
        else:
          compteur_composer[name]+=1
  except:
    pass

  dico_acteurs = dict(sorted(compteur_acteurs.items(), key=lambda x: x[1], reverse=True)[:500])
  dico_director = dict(sorted(compteur_director.items(), key=lambda x: x[1], reverse=True)[:500])
  dico_real = dict(sorted(compteur_real.items(), key=lambda x: x[1], reverse=True)[:500])
  dico_writer = dict(sorted(compteur_writer.items(), key=lambda x: x[1], reverse=True)[:500])
  dico_composer = dict(sorted(compteur_composer.items(), key=lambda x: x[1], reverse=True)[:500])

In [7]:
for acteur in dico_acteurs.keys():
  df[f"actor_{acteur}"] = zeros(df.shape[0])
for director in dico_director.keys():
  df[f"director_{director}"] = zeros(df.shape[0])
for real in dico_real.keys():
  df[f"real_{real}"] = zeros(df.shape[0])
for writer in dico_writer.keys():
  df[f"writer_{writer}"] = zeros(df.shape[0])
for composer in dico_composer.keys():
  df[f"composer_{composer}"] = zeros(df.shape[0])

C:\Users\valen\AppData\Local\Temp\ipykernel_14820\2378624390.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f"actor_{acteur}"] = zeros(df.shape[0])
C:\Users\valen\AppData\Local\Temp\ipykernel_14820\2378624390.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f"actor_{acteur}"] = zeros(df.shape[0])
C:\Users\valen\AppData\Local\Temp\ipykernel_14820\2378624390.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider

In [8]:
def remplir_donnees(row):
  try:
    names = eval(row["noms"]) # liste des noms des personnes qui ont travaillé dans le film
    jobs = eval(row["jobs"]) # leur job
    for name, job in zip(names, jobs):
      if (job == "actor" or job == "actress") and name in dico_acteurs:
        row[f"actor_{name}"] = 1
      if job == "director" and name in dico_director:
        row[f"director_{name}"] = 1
      if job == "real" and name in dico_real:
        row[f"real_{name}"] = 1
      if job == "writer" and name in dico_writer:
        row[f"writer_{name}"] = 1
      if job == "composer" and name in dico_composer:
        row[f"composer_{name}"] = 1
  except:
    pass
  return row

In [9]:
df = df.apply(remplir_donnees, axis=1)

In [10]:
def recup_nom(nom):
    for role in ["actor","real","actress","writer","composer","director"]:
        try:
            films = df[df[f"{role}_{nom}"]==1]
            display(films["originalTitle"])
        except:
            pass

In [11]:
recup_nom("Charlize Theron")

2 Days in the Valley_1996                                  2 Days in the Valley
That Thing You Do!_1996                                      That Thing You Do!
The Devil's Advocate_1997                                  The Devil's Advocate
Trial and Error_1997                                            Trial and Error
Mighty Joe Young_1998                                          Mighty Joe Young
The Cider House Rules_1999                                The Cider House Rules
The Astronaut's Wife_1999                                  The Astronaut's Wife
The Yards_2000                                                        The Yards
The Legend of Bagger Vance_2000                      The Legend of Bagger Vance
Reindeer Games_2000                                              Reindeer Games
Men of Honor_2000                                                  Men of Honor
Waking Up in Reno_2002                                        Waking Up in Reno
Sweet November_2001                     

In [12]:
df.head()

,Unnamed: 0,originalTitle,imdb_id,primaryTitle,averageRating,numVotes,titleType,startYear,runtimeMinutes,primaryName,...,composer_Shinichirô Ikebe,composer_Siu-Tin Lai,composer_Richard Harvey,composer_David Byrne,composer_Colin Towns,composer_Richard Robbins,composer_Howard Goodall,composer_Christopher L. Stone,composer_Anthony Marinelli,composer_Mark Suozzo
Partie de campagne_1946,0,Partie de campagne,tt0028445,A Day in the Country,7.5,7427,short,1946,40,"{'Jean Renoir', 'Pierre Lestringuez', 'Guy de ...",...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Kate & Leopold_2001,1,Kate & Leopold,tt0035423,Kate & Leopold,6.4,91790,movie,2001,118,"{'Bradley Whitford', 'Liev Schreiber', 'Brecki...",...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
The Best Years of Our Lives_1946,2,The Best Years of Our Lives,tt0036868,The Best Years of Our Lives,8.1,74474,movie,1946,170,"{'Myrna Loy', 'Virginia Mayo', 'Roman Bohnen',...",...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Anchors Aweigh_1945,3,Anchors Aweigh,tt0037514,Anchors Aweigh,7.0,9755,movie,1945,140,"{'Billy Gilbert', 'Joe Pasternak', 'Pamela Bri...",...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
And Then There Were None_1945,4,And Then There Were None,tt0037515,And Then There Were None,7.4,17602,movie,1945,97,"{'Walter Huston', 'Agatha Christie', 'Lucien N...",...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [59]:
df.to_csv("df_descriptif.csv")